# Aplicando o ferramental de tempo de passagem


In [19]:
import pandas as pd
import numpy as np

In [32]:
area2 = pd.read_csv('https://raw.githubusercontent.com/Gorgens/treelab/master/manejo/InventarioMultinivelArea2.csv')
area2['comum'] = area2['comum'].str.capitalize()
area2vivas = area2[area2['comum'] != 'Morta']

print(area2.shape, area2vivas.shape, area2vivas['parcela'].unique().shape)

(690, 10) (597, 10) (10,)


## Centro de classe

In [21]:
intervaloClasse = 10.0
diametroMin = 10.0
classeMax = ((max(area2vivas['dap.2013']) // intervaloClasse) + 1) * intervaloClasse

intervalos = list(np.arange(diametroMin, classeMax, intervaloClasse))
# intervalos = list(map((lambda x: x - 0.001), intervalos))
intervalos = [(x - 0.001) for x in intervalos]
centros = [round(x + intervaloClasse/2.0, 0) for x in intervalos[:-1]]

intervaloMedicao = 2

area2vivas = area2vivas[area2vivas['dap.2013'] >= diametroMin]
area2vivas['centroClasse'] = pd.cut(area2vivas['dap.2013'], bins=intervalos, labels=centros)
area2vivas.sample(10)

,parcela,arv,comum,cientifico,familia,dap.2013,Hcom.2013,Htot.2013,UTM_Easting,UTM_Northing,centroClasse
599,T09,11,Taxi branco,Tachigali paniculata,Fabaceae,72.4,21.05,26.00,219414.4178,9632694.825,75.0
235,T04,17,Fava arara,Parkia multijuga,Fabaceae,47.0,24.73,35.00,220729.7035,9633440.515,45.0
484,T07,21,Matamata,Eschweilera parviflora,Lecythidaceae,73.0,26.85,30.14,219170.8540,9633017.330,75.0
375,T05,57,Inga,Inga marginata,Fabaceae,38.7,16.48,28.57,220292.4148,9632601.592,35.0
303,T04,85,Quaruba,Vochysia guianensis,Vochysiaceae,11.0,9.60,14.60,220682.3622,9633416.669,15.0
54,T01,55,Inga vermelho,Inga alba,Fabaceae,34.0,10.11,17.67,220185.0421,9633831.213,35.0
86,T02,10,Angelim rajado,Zygia racemosa,Fabaceae,38.2,17.48,27.43,220479.5825,9633769.570,35.0
418,T06,9,Embauba,Cecropia ficifolia,Urticaceae,12.7,7.65,12.49,219032.2413,9633156.853,15.0
673,T10,43,Embauba branca,Cecropia palmata,Urticaceae,24.7,14.82,19.70,219617.6181,9632586.385,25.0
162,T03,27,Parapara,Jacaranda copaia,Bignoniaceae,39.7,14.80,22.72,220420.5298,9633535.592,35.0


## Modelo do tempo de passagem

In [22]:
# Modelo para estimar o incremento médio com base no centro de classe
def residenceTime(cc, a, b, c):
  return c / (1.+a * np.exp(-b*cc))

## Projetando um ano a frente

In [23]:
area2vivas['inc'] = area2vivas['centroClasse'].apply(lambda x: residenceTime(x, 14.4606, 0.1201, 0.6191))
area2vivas['inc'] = area2vivas['inc'].astype(float)
area2vivas['dap.2014est'] = area2vivas['dap.2013'] + area2vivas['inc']
area2vivas.sample(10)

,parcela,arv,comum,cientifico,familia,dap.2013,Hcom.2013,Htot.2013,UTM_Easting,UTM_Northing,centroClasse,inc,dap.2014est
132,T02,56,Macucu,Licania heteromorpha,Chrysobalanaceae,15.5,12.03,16.86,220576.6804,9633813.055,15.0,0.182801,15.682801
494,T07,31,Jatereu,Lecythis idatimon,Lecythidaceae,26.5,14.99,21.15,219235.1257,9633040.761,25.0,0.360329,26.860329
633,T10,3,Sucupira preta,Bowdichia nitida,Fabaceae,40.2,19.48,30.26,219300.3239,9632415.771,45.0,0.581304,40.781304
459,T06,50,Embauba,Cecropia ficifolia,Urticaceae,16.0,9.91,13.87,219333.0311,9633317.864,15.0,0.182801,16.182801
334,T05,15,Freijo branco,Cordia scabrifolia,Boraginaceae,10.6,4.17,7.88,220574.4859,9632751.739,15.0,0.182801,10.782801
580,T08,55,Matamata preto,Eschweilera ovata,Lecythidaceae,12.0,8.45,13.96,219516.2578,9632971.454,15.0,0.182801,12.182801
533,T08,8,Embauba,Cecropia ficifolia,Urticaceae,11.7,9.08,12.77,219151.2032,9632769.763,15.0,0.182801,11.882801
250,T04,32,Jatereu,Lecythis idatimon,Lecythidaceae,14.3,5.50,11.05,220606.3634,9633376.659,15.0,0.182801,14.482801
43,T01,44,Taxirana,Stryphnodendron paniculatum,Fabaceae,14.0,6.08,9.54,220116.8392,9633797.579,15.0,0.182801,14.182801
381,T05,63,Parapara,Jacaranda copaia,Bignoniaceae,59.8,22.01,35.34,220299.0111,9632598.356,55.0,0.607220,60.407220


In [24]:
temp = pd.DataFrame()
temp['dap1'] = area2vivas['dap.2013']
temp['dap0'] = area2vivas['dap.2013']
intervaloClasse = 10.0
diametroMin = 10.0
classeMax = ((max(temp['dap1']) // intervaloClasse) + 5) * intervaloClasse
intervalos = list(np.arange(diametroMin, classeMax, intervaloClasse))
intervalos = [(x - 0.001) for x in intervalos]
centros = [round(x + intervaloClasse/2.0, 0) for x in intervalos[:-1]]
  
t = 0
tmax = 30

while t < tmax: 
  
  temp['centroClasse'] = pd.cut(temp['dap1'], bins=intervalos, labels=centros)
  temp['inc'] = temp['centroClasse'].apply(lambda x: residenceTime(x, 14.4606, 0.1201, 0.6191))
  temp['inc'] = temp['inc'].astype(float)
  temp['dap2'] = temp['dap1'] + temp['inc']
  temp['dap1'] = temp['dap2']
  t += 1

temp.drop(['dap1'], axis=1, inplace=True)
temp.sample(10)

,dap0,centroClasse,inc,dap2
140,16.6,25.0,0.360329,24.036840
68,40.6,55.0,0.607220,58.376035
103,36.3,55.0,0.607220,53.291005
315,11.1,15.0,0.182801,16.584038
684,38.0,55.0,0.607220,55.357603
380,53.4,75.0,0.618005,71.778543
605,15.5,25.0,0.360329,21.871675
45,55.6,75.0,0.618005,74.013426
536,10.2,15.0,0.182801,15.684038
204,39.5,55.0,0.607220,57.151990


In [25]:
area2vivas.drop(['inc', 'dap.2014est'], axis=1, inplace=True)
area2vivas['dap30anos'] = temp['dap2']
area2vivas['centroClasse30anos'] = temp['centroClasse']
area2vivasCorMultinivel.sample(10)

,parcela,arv,comum,cientifico,familia,dap.2013,Hcom.2013,Htot.2013,UTM_Easting,UTM_Northing,centroClasse,dap30anos,centroClasse30anos
351,T05,32,Jatereu,Lecythis idatimon,Lecythidaceae,12.3,8.52,13.15,220465.3747,9632695.474,15.0,17.784038,15.0
607,T09,19,Embauba branca,Cecropia palmata,Urticaceae,11.5,8.73,11.59,219458.3140,9632720.835,15.0,16.984038,15.0
2,T01,3,Andorinha,Casearia grandiflora,Salicaceae,12.0,8.46,12.80,220493.8442,9634000.801,15.0,17.484038,15.0
130,T02,54,Abiurana branca,Pouteria guianensis,Sapotaceae,17.1,8.44,14.81,220582.9006,9633816.267,15.0,25.069422,25.0
656,T10,26,Burra leiteira,Sapium marmieri,Euphorbiaceae,12.0,8.02,12.00,219464.6720,9632503.766,15.0,17.484038,15.0
128,T02,52,Burra leiteira,Sapium marmieri,Euphorbiaceae,12.5,5.61,9.85,220591.9395,9633820.550,15.0,17.984038,15.0
664,T10,34,Embauba branca,Cecropia palmata,Urticaceae,15.5,10.37,13.03,219525.2928,9632536.583,15.0,21.871675,25.0
526,T08,1,Embauba,Cecropia ficifolia,Urticaceae,13.0,10.78,12.80,219122.8682,9632753.842,15.0,18.484038,15.0
408,T05,90,Taxi branco,Tachigali paniculata,Fabaceae,68.0,11.74,21.87,220714.3368,9632816.607,65.0,86.537696,85.0
628,T09,40,Inga,Inga marginata,Fabaceae,15.6,11.43,15.80,219621.0555,9632800.849,15.0,21.971675,25.0


In [26]:
# corrigir multinível
abaixo35 = area2vivas[area2vivas['dap.2013'] < 35.]
print(area2vivas.shape)
area2vivasCorMultinivel = pd.concat([area2vivas, abaixo35, abaixo35, abaixo35, abaixo35, abaixo35, abaixo35, abaixo35, abaixo35, abaixo35])
print(area2vivasCorMultinivel.shape)

(597, 13)
(3873, 13)


In [34]:
area2vivasCorMultinivel.pivot_table(values='arv', index='centroClasse', columns='centroClasse30anos', aggfunc='count', margins=True).div(10.)

centroClasse30anos,15.0,25.0,35.0,45.0,55.0,65.0,75.0,85.0,95.0,105.0,115.0,All
centroClasse,,,,,,,,,,,,
15.0,205.0,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298.0
25.0,NaN,NaN,40.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0
35.0,NaN,NaN,NaN,13.0,5.9,NaN,NaN,NaN,NaN,NaN,NaN,18.9
45.0,NaN,NaN,NaN,NaN,3.7,6.2,NaN,NaN,NaN,NaN,NaN,9.9
55.0,NaN,NaN,NaN,NaN,NaN,1.2,3.6,NaN,NaN,NaN,NaN,4.8
65.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.6,NaN,NaN,NaN,1.6
75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,0.3,NaN,NaN,0.6
85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.2,NaN,0.3
95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.1
